#### imports 

In [3]:
!pip install pandas
!pip install vexa-client

In [4]:
import os
import sys
import pandas as pd
from IPython.display import clear_output, display
import time

# Add the testing directory to the path
sys.path.append('../testing')

from core import create_user_client, request_bot,get_transcript

from vexa_client.vexa import parse_url


from dotenv import load_dotenv
load_dotenv()


True

#### single bot per user/meeting
#### concurrent_bots
1. create user with 2 concurrency
2. start 1 bot
3. request another bot for the same meeting immediately -> must fail
4. first bot -> must come, admit the bot
5. speak and test transcription
6. launch bot for another meeting, admit the bot, check trnascription
7. launch another bot for the third meeting -> must fail
8. stop bot meeting 2 and and launch bot 3, but come

#### stop bot works for requested and active meetings
1 start meeting 1, stop immediately, start meeting 2 ->bot must come


#### tests

In [5]:
meeting_urls = ["https://meet.google.com/xdy-txjo-fyh", 
            "https://meet.google.com/fjp-cbnn-hug",
            "https://meet.google.com/wjf-bhmw-bix"]

##### 1. create user with 2 concurrency

In [6]:
# creates test user and returns client
client = create_user_client(base_url=os.getenv('BASE_URL'),admin_api_key=os.getenv('ADMIN_API_TOKEN'),max_concurrent_bots=2)

In [ ]:
platform, native_meeting_id, passcode = parse_url(meeting_urls[0])

#2. start 1 bot

request_bot(client,platform, native_meeting_id, passcode)

#3. request another bot for the same meeting immediately (must fail)
try:request_bot(client,platform, native_meeting_id, passcode)
except Exception as e:
    print(e) #An active or requested meeting already exists for this platform and meeting ID

4. first bot must come, admit the bot and speak

In [ ]:
#time.sleep(15)
#5. speak and test transcription
get_transcript(client,platform, native_meeting_id,tail=50,duration=10)

In [ ]:
#6. launch bot for another meeting, admit the bot, check trnascription

platform, native_meeting_id, passcode = parse_url(meeting_urls[1])
request_bot(client,platform, native_meeting_id, passcode)


In [ ]:
#time.sleep(15)
get_transcript(client,platform, native_meeting_id)

In [ ]:
#7. launch another bot for the third meeting -> must fail
platform, native_meeting_id, passcode = parse_url(meeting_urls[2])
try:request_bot(client,platform, native_meeting_id, passcode)
except Exception as e:
    print(e) #User has reached the maximum concurrent bot limit (2).

In [ ]:
#8. stop bot meeting 2 and and launch bot 3, but come

platform, native_meeting_id, passcode = parse_url(meeting_urls[0])
client.stop_bot(platform=platform,native_meeting_id=native_meeting_id)

time.sleep(0.1)
platform, native_meeting_id, passcode = parse_url(meeting_urls[2])
request_bot(client,platform, native_meeting_id, passcode)

In [ ]:
running_bots = client.get_running_bots_status()
len(running_bots)


In [ ]:
meetings = client.get_meetings()
[m['status'] for m in meetings]

In [ ]:
meetings

In [ ]:
for m in meeting_urls:
    try:
        platform, native_meeting_id, passcode = parse_url(m)
        client.stop_bot(platform=platform,native_meeting_id=native_meeting_id)
    except Exception as e:
        print(e)

In [ ]:
meetings = client.get_meetings()
[m['status'] for m in meetings] == ['completed', 'completed', 'completed']